In [81]:
import os
import pandas as pd
import requests
import json

from elasticsearch import Elasticsearch

In [121]:
# Get players

players = pd.read_json("game-changers/esports-data/players.json")
teams = pd.read_json("game-changers/esports-data/teams.json")

#players = pd.merge(players, teams[['id', 'name', 'slug', 'acronym']], left_on='home_team_id', right_on='id', how='left')

# Rename columns to clean up
#players.rename(columns={'name': 'team_name', 'slug': 'team_slug', 'acronym': 'team_acronym'}, inplace=True)

players

,id,handle,first_name,last_name,status,photo_url,home_team_id,created_at,updated_at
0,107025876564296048,Toma,Tommi,Lehtinen,active,http://static.lolesports.com/players/163303303...,106652238530500032,2021-10-01 11:19:00+00:00,2021-10-01 11:19:00+00:00
1,106977714184830000,Stefanie,Stefanie,Jones,active,http://static.lolesports.com/players/163235037...,106976771652907808,2021-09-22 22:41:26+00:00,2023-04-09 14:50:55+00:00
2,107176790303905248,aRth,Arthur,Hardman,active,None,107174817570297248,2021-10-28 14:16:57+00:00,2021-11-01 13:52:31+00:00
3,107282855606202832,godwana,Engin,Balcı,active,http://static.lolesports.com/players/163700646...,107021298845350512,2021-11-15 21:00:23+00:00,2023-05-29 08:05:32+00:00
4,106977742188901808,Jaxsen,Jackson,Popelka,active,http://static.lolesports.com/players/163235080...,106976774374552352,2021-09-22 22:49:42+00:00,2021-09-22 22:49:42+00:00
...,...,...,...,...,...,...,...,...,...
2994,108764513807403456,R4M,Tokushima,Shiina,active,http://static.lolesports.com/players/164373420...,108764409368519856,2024-07-08 03:07:11+00:00,2024-07-08 03:07:11+00:00
2995,106371654690991520,bizerra,Vitória,Bezerra,active,None,108453893686167872,2024-07-16 02:06:58+00:00,2024-07-16 02:06:58+00:00
2996,112793696105404256,WINT3R,winter,winter,active,https://s3.us-west-2.amazonaws.com/static.lole...,112251458972768976,2024-07-16 02:09:10+00:00,2024-07-21 14:04:21+00:00
2997,112251720337473920,natasha05,Giulia,Natasha,active,https://s3.us-west-2.amazonaws.com/static.lole...,109993966808522032,2024-07-20 00:39:24+00:00,2024-07-20 00:39:24+00:00


In [ ]:
players.dtypes

In [ ]:
107025876564296048

In [136]:
players[players['id'] == 107115843208519317]

,id,handle,first_name,last_name,status,photo_url,home_team_id,created_at,updated_at


In [137]:
players[players['handle'] == "CaspeR"]

,id,handle,first_name,last_name,status,photo_url,home_team_id,created_at,updated_at
20,107115843208519312,CaspeR,Neha,Sottany,active,None,108076630730769008,2022-09-21 10:05:39+00:00,2023-03-13 06:23:10+00:00
155,107115843208519312,CaspeR,Neha,Sottany,active,None,108076630730769008,2023-03-13 06:25:28+00:00,2023-05-30 08:22:02+00:00
200,107115843208519312,CaspeR,Neha,Sottany,active,None,108076630730769008,2022-09-21 10:05:38+00:00,2022-09-21 10:05:39+00:00
968,107115843208519312,CaspeR,Neha,Sottany,active,None,108076630730769008,2023-05-30 08:27:42+00:00,2024-08-22 12:27:17+00:00
1134,107115843208519312,CaspeR,Neha,Sottany,active,None,108076630730769008,2022-04-05 01:35:14+00:00,2022-09-21 10:05:38+00:00
1502,107115843208519312,CaspeR,Neha,Sottany,active,None,107115766504608544,2021-10-17 08:25:25+00:00,2022-04-05 01:35:14+00:00
2433,107115843208519312,CaspeR,Neha,Sottany,archived,None,108076630730769008,2024-08-22 12:27:17+00:00,2024-08-22 12:27:17+00:00


In [ ]:
# Get all player ids for a player handle
target = "rayse"

target_ids = list(set(players[players['handle'] == target]['id']))
target_ids

In [140]:
# Get games for a tournament that involves the player
player_id = "107115843208519317" # rayse

games = pd.read_json("game-changers/esports-data/mapping_data.json")

games

,platformGameId,esportsGameId,tournamentId,teamMapping,participantMapping
0,val:a02b939b-d53b-4d6c-9cf9-8df5cbecdec6,110614988238486656,110614878961138672,"{'18': '107185970379663407', '19': '1082407634...","{'1': '110196391536301033', '2': '108550500823..."
1,val:865eb670-8fd8-4245-97b3-44540e321f8e,110780564307690144,110780564306117264,"{'18': '110745562660975168', '19': '1087481697...","{'1': '108748099637064824', '2': '108748093351..."
2,val:4d834f45-c9db-42fa-ae55-e8afd451e434,109879965968343856,109879965964411664,"{'17': '108452390946539707', '18': '1076008816...","{'1': '107600796790972557', '2': '107600798796..."
3,val:178e8b84-61fa-4a14-835d-8f0e0af83466,111006223833455232,111006194588670528,"{'19': '108748177717149112', '20': '1110572219...","{'1': '111067926063836716', '2': '111067931077..."
4,val:03bdf51f-453c-4fab-9406-63beee427e6d,109879965968475072,109879965964411664,"{'17': '108172266878972669', '18': '1098798864...","{'1': '107897747787068260', '2': '107910792460..."
...,...,...,...,...,...
1098,val:d507ec8f-8ab8-4a65-94d6-64f34aa8010e,112472465715573328,112472465714786864,"{'17': '111901707633760155', '18': '1071859703...","{'1': '107025879556684669', '2': '107025879465..."
1099,val:945b723a-0f6d-4243-9d80-24013f39139c,112196265094407568,112093580768495968,"{'16': '112172170311847035', '17': '1121843686...","{'1': '112184652059037339', '2': '112184651995..."
1100,val:77e3feca-b86d-43a2-b01c-3758d16ee7d3,112279699443897088,112279699443110656,"{'18': '108748177717149112', '19': '1087481697...","{'1': '111057371424123924', '2': '108748099637..."
1101,val:0b9b55b0-979b-4c25-8033-8ccdc7a8f451,112279699443897088,112279699443110656,"{'19': '112279706800846545', '20': '1107455626...","{'1': '111057427541732898', '2': '108748092183..."


In [141]:
player_games = []

games_filter = games[games['participantMapping'].apply(lambda mapping: player_id in mapping.values())]

# Remove prefix
games_filter['platformGameId'] = games_filter['platformGameId'].apply(lambda gameID: gameID[4:])

player_games = list(games_filter['platformGameId'])


player_games

C:\Users\Angus\AppData\Local\Temp\ipykernel_19092\3499574545.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_filter['platformGameId'] = games_filter['platformGameId'].apply(lambda x: x[4:])


['ecaa1621-bb56-4daa-9cdf-23571c0cd86e',
 'c97e42fc-eaa7-4aa9-be84-7ada3f18951d',
 '34b0c851-5e26-45bf-95f0-9af48b228834',
 '0be4d5e2-5e95-40c8-8ade-7b98ca6c24d4',
 '4f3778a8-1bb9-43d5-bf49-a8042e7d0a5f',
 'ea001b85-620c-4e1b-8b0c-8f595a8c4f2e',
 'ead8ca70-f4aa-4c1b-85ae-aad88856422d',
 '28246ecf-2eb6-41cb-96eb-9d3d08d3cb40',
 'a8054c74-10ed-4896-a2ac-b8eb8765d552',
 'eb8f3d48-6ad9-4c0d-b01a-51930544a89a',
 'ca7fac1d-e1fa-4684-895b-bd5039ad09b9',
 '40b9e05e-88d9-481b-a01a-c0468a17e4e7',
 'c62f1e96-4d6c-4115-aefb-fffe492f6c56',
 '837a7a1f-cb5c-435d-afb6-37b8f33571a1',
 '141ea61c-6745-4095-aa61-f01187518ddb',
 'c5cd9cdc-b7ad-4442-a2f3-59c81816105b',
 'e8282a76-dd55-4701-a630-ec7f5715e3fa',
 '77a60e32-8065-48b5-a4c3-62c40232ae58',
 '81302c63-0267-4baa-a958-db96e4b8ebda',
 'd993921f-dec9-4daa-a46e-1a8d5da80917',
 'bbbee58d-e6c0-4522-be67-1ab9a3facb64',
 '7726a423-57a9-4a28-b871-0689d6f9732e',
 'e3facd47-f9bf-44f7-9f44-a439fddc7453',
 '1ba420f6-63d0-4fe0-a4f2-bf77d5f4f553',
 '1ab1fd35-d39b-

In [142]:
len(player_games)

30

In [143]:
# Create map of who downed who in a target game
target_game = "ecaa1621-bb56-4daa-9cdf-23571c0cd86e"

try:
        with open(os.path.join('game-changers/games/2022/', 'val_'+target_game+'.json'), 'r') as file:

                game_data = json.load(file)

except:
        print("No such file")

In [144]:
# Get player mapping to connect player id 
game_player_map = games[games['platformGameId'] == "val:"+target_game]['participantMapping']
game_player_map = game_player_map.to_dict()[game_player_map.index[0]]
game_player_map

{'1': '107115843208519317',
 '2': '107875217699783931',
 '3': '107115944851316499',
 '4': '107115867411816133',
 '5': '108076829264740999',
 '6': '107115874551533079',
 '7': '107115873869368850',
 '8': '107115875415085942',
 '9': '107875219391944954',
 '10': '107875219267798319'}

In [145]:
# Get player deaths

deaths_from = {}

for snapshot in game_data:

    if "playerDied" in snapshot:
        print(snapshot)

        # Check if death is our target player
        died_player = str(snapshot['playerDied']['deceasedId']['value'])
        died_player_id = game_player_map[died_player]

        if died_player_id == player_id:
                

            killer = str(snapshot['playerDied']['killerId']['value'])
            killer_player_id = game_player_map[killer]

            # Check if deaths from player already exist
            if killer_player_id in deaths_from:
                deaths_from[killer_player_id] += 1
            else:
                deaths_from[killer_player_id] = 1


{'platformGameId': 'val:ecaa1621-bb56-4daa-9cdf-23571c0cd86e', 'playerDied': {'deceasedId': {'value': 7}, 'assistants': [], 'killerId': {'value': 1}}, 'metadata': {'serverInfo': {'processId': '', 'rfc190Scope': 'aresriot.aws-apse1-prod.tournament-gp-singapore-1.ares.gameserver'}, 'playback': 0, 'sequenceNumber': 650, 'stage': 1, 'gameVersion': '05.06.00.765767', 'gameId': {'value': 'ecaa1621-bb56-4daa-9cdf-23571c0cd86e'}, 'eventTime': {'includedPauses': '508.814s', 'omittingPauses': '508.814s'}, 'wallTime': '2022-10-02T14:16:17.384Z'}}
{'platformGameId': 'val:ecaa1621-bb56-4daa-9cdf-23571c0cd86e', 'metadata': {'serverInfo': {'processId': '', 'rfc190Scope': 'aresriot.aws-apse1-prod.tournament-gp-singapore-1.ares.gameserver'}, 'playback': 0, 'sequenceNumber': 659, 'stage': 1, 'gameVersion': '05.06.00.765767', 'gameId': {'value': 'ecaa1621-bb56-4daa-9cdf-23571c0cd86e'}, 'eventTime': {'includedPauses': '513.243s', 'omittingPauses': '513.243s'}, 'wallTime': '2022-10-02T14:16:21.813Z'}, 'pla

In [146]:

print(deaths_from)

{'107115873869368850': 4, '107875219391944954': 3, '107115875415085942': 2}


In [ ]:
game_player_map['1']

In [ ]:
# Add elastisearch for searching
games_dir = 'J:/VCT Hackathon Val Tournament Data/game-changers/games/2022/'

search = Elasticsearch()

matching = []

for game in os.listdir(games_dir):

    with open(os.path.join(games_dir, game), 'r') as file:

        game_data = json.load(file)

        search.index(index='json_files', body=game_data)